In [521]:
#News or Noise - Algo Trading Signals

In [522]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets
import alpaca_trade_api as tradeapi
import quandl
import os
import requests
import ccxt
import time

In [523]:
# Pull in price data for ETF's from Alpaca API from 1/1/2010 - 12/31/2020
# SPY - S&P 500
# XLF - Financials Sector
# XLK - Technology
# XTL - Telecom Sector
# XLU - Utilities
# XLI - Industrials
# XLV - Healthcare
# XLY - Consumer Discretionary
# XLP - Consumer Staples
# XLB - Materials
# XOP - Oil & Gas
# XHB - Homebuilders
# XME - Metals and Miners
# XRT - Retail


In [524]:
load_dotenv()

True

In [525]:
#import

api_key_QD = os.getenv("QUANDL")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# type(api_key_QD)
# type(alpaca_api_key)
print(alpaca_api_key)

PKZIM8DTCXGYC49VFLLC


In [526]:
#import alpaca api file for ETF price history
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2010-01-02", tz="America/New_York").isoformat()
today = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["SPY", "XLF", "XLK","XLU","XLI", "XLV", "XLY", "XLP", "XLB", "XOP", "XHB", "XME", "XRT", "ETH", "LTC"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get closing prices
df_ETFs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = today
).df

# Preview DataFrame
# YOUR CODE HERE!
df_ETFs.head()

# Output the data to CSV
df_ETFs.to_csv("ETFs1.csv", encoding='utf-8', index=True)

In [540]:
# Set path to CSV and read in CSV
ETF_csv = Path("ETFs.csv")
# Set index as datetime object and drop extraneous columns
ETF_df = pd.read_csv(ETF_csv, index_col='Unnamed: 0', infer_datetime_format=True, parse_dates=True)
ETF_df = ETF_df.drop(ETF_df.index[0])
ETF_df.index = pd.to_datetime(ETF_df.index, utc=True).date
ETF_df.sort_index(inplace=True)

#drop unneccessary columns and keep close price columns
ETF_close_prices_df = ETF_df.loc[:, ['SPY.3', 'XLF.3', 'XLK.3', 'XLU.3', 'XLI.3', 'XLV.3',
                                    'XLY.3', 'XLP', 'XLB.3', 'XOP.3', 'XHB.3', 'XME.3', 'XRT.3', 'ETH.3', 'LTC.3']]
ETF_close_prices_df.rename(columns={'SPY.3': 'SPY',
                   'XLF.3': 'XLF',
                   'XLK.3': 'XLK',
                   'XLU.3': 'XLU',
                   'XLI.3': 'XLI',
                   'XLV.3': 'XLV',
                   'XLY.3': 'XLY',
                   'XLP.3': 'XLP',
                   'XLB.3': 'XLB',
                   'XOP.3': 'XOP',
                   'XHB.3': 'XHB',
                   'XME.3': 'XME',
                   'XRT.3': 'XRT',
                   'ETH.3': 'ETH',
                   'LTC.3': 'LTC'
                  }, inplace=True)

ETF_close_prices_df[['SPY', 'XLF', 'XLK', 'XLU', 'XLI', 
                     'XLV', 'XLY', 'XLP', 'XLB', 'XOP', 'XHB', 'XME', 'XRT', 'ETH', 'LTC']] = ETF_close_prices_df[['SPY', 'XLF', 'XLK', 'XLU', 'XLI', 
                     'XLV', 'XLY', 'XLP', 'XLB', 'XOP', 'XHB', 'XME', 'XRT', 'ETH', 'LTC']].astype(float)
ETF_close_prices_df.dtypes

#calculate daily returns
ETF_daily_returns_df = ETF_close_prices_df.pct_change().dropna()
ETF_close_prices_df.dropna(inplace=True)
ETF_close_prices_df.describe()

,SPY,XLF,XLK,XLU,XLI,XLV,XLY,XLP,XLB,XOP,XHB,XME,XRT,ETH,LTC
count,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000,2766.000000
mean,207.600732,21.689694,49.359508,45.286939,52.255850,64.947741,76.575931,46.141299,47.006211,45.743315,32.034322,36.793337,38.626749,23.846027,39.121966
std,67.237094,4.950859,25.039529,10.234608,18.483666,23.778199,32.050182,11.235168,9.640819,14.261370,10.123195,13.377835,9.881248,5.918813,7.656156
min,102.260000,11.270000,20.300000,28.091300,22.390000,27.950000,28.620000,25.530000,28.060000,7.510000,12.530000,11.980000,8.000000,8.610000,21.670000
25%,141.525000,16.410000,29.546250,36.442875,32.670000,39.582500,46.395000,35.615000,38.500000,35.150000,25.017500,27.332500,30.682500,19.482500,33.070000
50%,205.120000,22.920000,41.977500,44.300000,51.385000,69.430000,75.965000,49.120000,47.400000,43.595000,33.320000,34.190000,41.881500,24.195000,39.775000
75%,265.207500,25.060000,65.725000,53.050000,69.970000,83.602500,102.207500,54.720000,54.667500,56.087500,38.590000,43.302500,45.057500,28.650000,45.480000
max,373.850000,31.165000,130.520000,70.970000,89.670000,113.430000,160.770000,68.150000,72.380000,83.430000,59.410000,76.490000,92.440000,38.800000,54.200000


In [528]:
# Pull in crypto currency price data from Krakken API from 1/1/2010 - 12/31/2020
# BTC
# ETH
# LTC